# VAR(2) Model - DoD Bureaucratic Growth Analysis v12.3**Purpose**: Full Vector Autoregression with 2 lags on 8 selected variables.**Dataset**: `complete_normalized_dataset_v12.3.xlsx`**Analysis Steps**:1. Load 8 selected variables2. Test stationarity (ADF tests)3. Estimate VAR(2) model4. Extract coefficients5. Run Granger causality tests6. Compute Impulse Response Functions7. Compute Forecast Error Variance Decomposition---

## Setup and Configuration

In [ ]:
import pandas as pdimport numpy as npimport matplotlib.pyplot as pltimport seaborn as snsfrom statsmodels.tsa.api import VARfrom statsmodels.tsa.stattools import adfullerfrom scipy import statsfrom pathlib import Pathimport warningswarnings.filterwarnings('ignore')# ConfigurationDATA_FILE = '../complete_normalized_dataset_v12.3.xlsx'OUTPUT_DIR = '.'LAG_ORDER = 2# 8 variables from Granger causality analysisSELECTED_VARS = [    'Warrant_Officers_Z',    'Policy_Count_Log',    'Company_Grade_Officers_Z',    'Total_PAS_Z',    'FOIA_Simple_Days_Z',    'Junior_Enlisted_Z',    'Field_Grade_Officers_Z',    'Total_Civilians_Z']print("=" * 100)print(f"VAR({LAG_ORDER}) MODEL ANALYSIS - v12.3 DATASET")print("8 Variables Selected from Pairwise Granger Causality")print("=" * 100)

## 1. Load and Prepare Data

In [ ]:
print("\n[1/7] Loading data...")df = pd.read_excel(DATA_FILE)data = df[SELECTED_VARS].dropna()print(f"  Observations: {len(data)}")print(f"  Variables: {len(SELECTED_VARS)}")print("\n  Selected variables:")for i, var in enumerate(SELECTED_VARS, 1):    print(f"    {i}. {var}")

## 2. Test Stationarity (ADF Tests)

In [ ]:
print("\n[2/7] Testing stationarity (ADF tests)...")stationarity_results = []for var in SELECTED_VARS:    adf_result = adfuller(data[var], autolag='AIC')    stationarity_results.append({        'Variable': var,        'ADF_Statistic': adf_result[0],        'p_value': adf_result[1],        'Lags_Used': adf_result[2],        'Stationary': 'Yes' if adf_result[1] < 0.05 else 'No'    })stationarity_df = pd.DataFrame(stationarity_results)stationarity_df.to_excel(f'{OUTPUT_DIR}/stationarity_tests.xlsx', index=False)print("\n  Stationarity Test Results:")print("  " + "-" * 80)for _, row in stationarity_df.iterrows():    status = "[STATIONARY]" if row['Stationary'] == 'Yes' else "[NON-STATIONARY]"    print(f"    {row['Variable']:30s} ADF={row['ADF_Statistic']:8.4f}, p={row['p_value']:.4f} {status}")stationary_count = (stationarity_df['Stationary'] == 'Yes').sum()print(f"\n  Stationary variables: {stationary_count}/{len(SELECTED_VARS)}")

## 3. Estimate VAR(2) Model

In [ ]:
print(f"\n[3/7] Estimating VAR({LAG_ORDER}) model...")model = VAR(data)var_result = model.fit(maxlags=LAG_ORDER, ic=None)print(f"\n  Model estimated successfully")print(f"  Lag order: {var_result.k_ar}")print(f"  Number of equations: {var_result.neqs}")print(f"  Number of coefficients per equation: {var_result.k_ar * var_result.neqs + 1}")print(f"  Total observations used: {var_result.nobs}")

## 4. Model Diagnostics and R-squared

In [ ]:
print(f"\n[4/7] Generating model summary and diagnostics...")# Calculate R-squared for each equationrsquared_data = []residuals = var_result.resid.values if hasattr(var_result.resid, 'values') else var_result.residfor i, var in enumerate(SELECTED_VARS):    resid = residuals[:, i]    y_actual = data[var].values[-len(resid):]    y_pred = y_actual - resid        ss_res = np.sum(resid ** 2)    ss_tot = np.sum((y_actual - np.mean(y_actual)) ** 2)    r_squared = 1 - (ss_res / ss_tot) if ss_tot != 0 else 0        n = len(resid)    k = LAG_ORDER * len(SELECTED_VARS) + 1    adj_r_squared = 1 - (1 - r_squared) * (n - 1) / (n - k) if (n - k) > 0 else 0        rsquared_data.append({        'Variable': var,        'R_squared': r_squared,        'Adj_R_squared': adj_r_squared    })rsquared_df = pd.DataFrame(rsquared_data)rsquared_df.to_excel(f'{OUTPUT_DIR}/model_fit_rsquared.xlsx', index=False)print("\n  R-squared by equation:")print("  " + "-" * 60)for _, row in rsquared_df.iterrows():    print(f"    {row['Variable']:30s} R2={row['R_squared']:.4f}, Adj-R2={row['Adj_R_squared']:.4f}")avg_rsq = rsquared_df['R_squared'].mean()print(f"\n  Average R-squared: {avg_rsq:.4f}")

## 5. Extract Coefficient Matrices

In [ ]:
print(f"\n[5/7] Extracting coefficient matrices...")for lag in range(1, LAG_ORDER + 1):    coef_matrix = pd.DataFrame(index=SELECTED_VARS, columns=SELECTED_VARS)        for to_var in SELECTED_VARS:        for from_var in SELECTED_VARS:            param_name = f'L{lag}.{from_var}'            if param_name in var_result.params.index:                coef_matrix.loc[from_var, to_var] = var_result.params.loc[param_name, to_var]            else:                coef_matrix.loc[from_var, to_var] = 0.0        coef_matrix = coef_matrix.astype(float)    coef_matrix.to_excel(f'{OUTPUT_DIR}/coefficients_lag{lag}.xlsx')    print(f"  Saved coefficients for lag {lag}")# Save constant termsconst_df = pd.DataFrame({    'Equation': SELECTED_VARS,    'Constant': [var_result.params.loc['const', var] for var in SELECTED_VARS]})const_df.to_excel(f'{OUTPUT_DIR}/constants.xlsx', index=False)print(f"  Saved constant terms")

## 6. Granger Causality Tests

In [ ]:
print(f"\n[6/7] Running Granger causality tests on fitted model...")granger_results = []for caused_var in SELECTED_VARS:    for causing_var in SELECTED_VARS:        if caused_var == causing_var:            continue                try:            gc_test = var_result.test_causality(caused_var, causing_var, kind='f')            granger_results.append({                'Caused': caused_var,                'Causing': causing_var,                'F_statistic': gc_test.test_statistic,                'p_value': gc_test.pvalue,                'df_num': gc_test.df,                'df_denom': gc_test.df_denom,                'Significant_5pct': gc_test.pvalue < 0.05,                'Significant_1pct': gc_test.pvalue < 0.01            })        except:            continuegranger_causality_df = pd.DataFrame(granger_results)if len(granger_causality_df) > 0:    granger_causality_df = granger_causality_df.sort_values('p_value')    granger_causality_df.to_excel(f'{OUTPUT_DIR}/granger_causality_tests.xlsx', index=False)        sig_5pct = granger_causality_df['Significant_5pct'].sum()    sig_1pct = granger_causality_df['Significant_1pct'].sum()    print(f"\n  Granger causality test results:")    print(f"    Total tests: {len(granger_causality_df)}")    print(f"    Significant at 5%: {sig_5pct}")    print(f"    Significant at 1%: {sig_1pct}")else:    print(f"\n  No Granger causality tests completed successfully")    sig_5pct = 0    sig_1pct = 0

## 7. Impulse Response Functions and FEVD

In [ ]:
print(f"\n[7/7] Computing impulse response functions...")# Compute IRFs (10 periods ahead)irf = var_result.irf(10)# Plot IRFstry:    fig = irf.plot(orth=False, figsize=(24, 20))    plt.suptitle('Impulse Response Functions (Non-Orthogonalized) - All Variables',                 fontsize=16, fontweight='bold', y=0.995)    plt.tight_layout(rect=[0, 0, 1, 0.99])    plt.savefig(f'{OUTPUT_DIR}/impulse_response_functions_all.png', dpi=300, bbox_inches='tight')    plt.close()    print("  IRF plots saved")except Exception as e:    print(f"  Warning: Could not plot IRFs: {e}")# Save IRF datairf_data = []for i, impulse_var in enumerate(SELECTED_VARS):    for j, response_var in enumerate(SELECTED_VARS):        irf_values = irf.irfs[:, j, i]        for period in range(len(irf_values)):            irf_data.append({                'Impulse': impulse_var,                'Response': response_var,                'Period': period,                'IRF_Value': irf_values[period]            })irf_df = pd.DataFrame(irf_data)irf_df.to_excel(f'{OUTPUT_DIR}/impulse_response_data.xlsx', index=False)print("  IRF data saved")# FEVDprint("\n  Computing forecast error variance decomposition...")fevd = var_result.fevd(10)for i, var in enumerate(SELECTED_VARS):    fevd_var = pd.DataFrame(        fevd.decomp[:, i, :],        columns=SELECTED_VARS    )    fevd_var.insert(0, 'Period', range(len(fevd_var)))    fevd_var.to_excel(f'{OUTPUT_DIR}/fevd_{var}.xlsx', index=False)print("  FEVD saved for all variables")

## Summary

In [ ]:
print("\n" + "=" * 100)print(f"VAR({LAG_ORDER}) ANALYSIS COMPLETE")print("=" * 100)print(f"\nMODEL SPECIFICATION:")print(f"  Lag order: {LAG_ORDER}")print(f"  Number of variables: {len(SELECTED_VARS)}")print(f"  Observations used: {var_result.nobs}")print(f"  Total parameters: {len(SELECTED_VARS) * (len(SELECTED_VARS) * LAG_ORDER + 1)}")print(f"\nMODEL FIT:")print(f"  Average R-squared: {avg_rsq:.4f}")print(f"  AIC: {var_result.aic:.2f}")print(f"  BIC: {var_result.bic:.2f}")print(f"  HQIC: {var_result.hqic:.2f}")print(f"\nDIAGNOSTICS:")print(f"  Stationary variables: {stationary_count}/{len(SELECTED_VARS)}")print(f"  Significant Granger causalities (5%): {sig_5pct}/{len(granger_causality_df)}")print("=" * 100)

## 8. Network Analysis - Create Adjacency Matrices

Generate network representations from the VAR(2) coefficients to visualize variable relationships.

In [ ]:
print("
[8/8] Creating network adjacency matrices...")

# Load coefficient matrices
coef_lag1 = pd.read_excel('coefficients_lag1.xlsx', index_col=0)
coef_lag2 = pd.read_excel('coefficients_lag2.xlsx', index_col=0)

# Combined network (sum of absolute values)
adjacency_abs = np.abs(coef_lag1.values) + np.abs(coef_lag2.values)
adjacency_abs_df = pd.DataFrame(adjacency_abs, index=SELECTED_VARS, columns=SELECTED_VARS)
adjacency_abs_df.to_excel('network_adjacency_matrix_combined.xlsx')
print("  Saved: network_adjacency_matrix_combined.xlsx")

# Signed network (preserving direction)
adjacency_signed = coef_lag1.values + coef_lag2.values
adjacency_signed_df = pd.DataFrame(adjacency_signed, index=SELECTED_VARS, columns=SELECTED_VARS)
adjacency_signed_df.to_excel('network_adjacency_matrix_signed.xlsx')
print("  Saved: network_adjacency_matrix_signed.xlsx")

# Create edge list
edges = []
threshold = 0.05

for i, from_var in enumerate(SELECTED_VARS):
    for j, to_var in enumerate(SELECTED_VARS):
        if i == j:
            continue

        coef_t1 = coef_lag1.iloc[i, j]
        coef_t2 = coef_lag2.iloc[i, j]
        magnitude = abs(coef_t1) + abs(coef_t2)

        if magnitude > threshold:
            signed_sum = coef_t1 + coef_t2
            direction = 'Amplifying' if signed_sum > 0 else 'Dampening'

            edges.append({
                'From': from_var,
                'To': to_var,
                'Lag1_Coef': coef_t1,
                'Lag2_Coef': coef_t2,
                'Combined_Magnitude': magnitude,
                'Signed_Sum': signed_sum,
                'Direction': direction
            })

edges_df = pd.DataFrame(edges)
edges_df = edges_df.sort_values('Combined_Magnitude', ascending=False)
edges_df.to_excel('network_edge_list.xlsx', index=False)

print(f"  Saved: network_edge_list.xlsx ({len(edges_df)} edges)")
print(f"  Threshold: |coef| > {threshold}")

# Display top relationships
print("
TOP 10 STRONGEST RELATIONSHIPS:")
print("-" * 80)
for idx, row in edges_df.head(10).iterrows():
    print(f"  {row['From']:30s} -> {row['To']:30s}: {row['Combined_Magnitude']:6.3f} ({row['Direction']})")

edges_df.head(10)

## Network Statistics

In [ ]:
total_possible = len(SELECTED_VARS) * (len(SELECTED_VARS) - 1)
density = len(edges_df) / total_possible

print("
NETWORK STATISTICS:")
print("=" * 80)
print(f"Nodes (variables): {len(SELECTED_VARS)}")
print(f"Edges (relationships > {threshold}): {len(edges_df)}")
print(f"Total possible directed edges: {total_possible}")
print(f"Network density: {density:.3f}")

# Degree centrality
in_degree = edges_df['To'].value_counts()
out_degree = edges_df['From'].value_counts()

print("
Most Influenced (In-Degree):")
for var, count in in_degree.head(5).items():
    print(f"  {var:40s}: {count} incoming edges")

print("
Most Influential (Out-Degree):")
for var, count in out_degree.head(5).items():
    print(f"  {var:40s}: {count} outgoing edges")